In [2]:
from tensorflow import keras
import tensorflow as tf

## 간단한 모델 만들기
### Sequential 모델
* 케라스는 층(Layer)을 조합하여 모델(model)을 만든다.

In [3]:
from tensorflow.keras import layers

model = tf.keras.Sequential()
# 64개의 유닛을 가진 완전 연결 층을 모델에 추가합니다:
model.add(layers.Dense(64, activation='relu'))
# 또 하나를 추가합니다:
model.add(layers.Dense(64, activation='relu'))
# 10개의 출력 유닛을 가진 소프트맥스 층을 추가합니다:
model.add(layers.Dense(10, activation='softmax'))

### 층 설정

In [4]:
# 시그모이드 활성화 층을 만듭니다:
layers.Dense(64, activation='sigmoid')

# 또는 다음도 가능합니다:
layers.Dense(64, activation=tf.keras.activations.sigmoid)

# 커널 행렬에 L1 규제가 적용된 선형 활성화 층. 하이퍼파라미터 0.01은 규제의 양을 조절합니다:
layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l1(0.01))

# 절편 벡터에 L2 규제가 적용된 선형 활성화 층. 하이퍼파라미터 0.01은 규제의 양을 조절합니다:
layers.Dense(64, bias_regularizer=tf.keras.regularizers.l2(0.01))

# 커널을 랜덤한 직교 행렬로 초기화한 선형 활성화 층:
layers.Dense(64, kernel_initializer='orthogonal')

# 절편 벡터를 상수 2.0으로 설정한 선형 활성화 층:
layers.Dense(64, bias_initializer=tf.keras.initializers.Constant(2.0))

## 훈련과 평가
### 훈련 준비

In [5]:
model = tf.keras.Sequential([
# 64개의 유닛을 가진 완전 연결 층을 모델에 추가합니다:
layers.Dense(64, activation='relu', input_shape=(32,)),
# 또 하나를 추가합니다:
layers.Dense(64, activation='relu'),
# 10개의 출력 유닛을 가진 소프트맥스 층을 추가합니다:
layers.Dense(10, activation='softmax')])

모델 훈련을 설정하는 몇 가지 예시

In [7]:
# 평균 제곱 오차로 회귀 모델을 설정합니다.
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='mse',       # 평균 제곱 오차
              metrics=['mae'])  # 평균 절댓값 오차

# 크로스엔트로피 손실 함수로 분류 모델을 설정합니다.
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_10 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_11 (Dense)             (None, 10)                650       
Total params: 6,922
Trainable params: 6,922
Non-trainable params: 0
_________________________________________________________________


데이터셋이 작은 경우 `numpy`배열을 메모리에 적제하여 모델을 훈련하고 평가

In [11]:
import numpy as np

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

val_data = np.random.random((100, 32))
val_labels = np.random.random((100, 10))

model.fit(data, labels, epochs=10, batch_size=32,
          validation_data=(val_data, val_labels))

Train on 1000 samples, validate on 100 samples
Epoch 1/10
1000/1000 [==============================] - 0s 182us/sample - loss: 24633.4509 - categorical_accuracy: 0.0930 - val_loss: 32977.4253 - val_categorical_accuracy: 0.0900
Epoch 2/10
1000/1000 [==============================] - 0s 95us/sample - loss: 28666.3931 - categorical_accuracy: 0.0880 - val_loss: 33430.6459 - val_categorical_accuracy: 0.1000
Epoch 3/10
1000/1000 [==============================] - 0s 91us/sample - loss: 32619.4897 - categorical_accuracy: 0.0950 - val_loss: 52887.2494 - val_categorical_accuracy: 0.1000
Epoch 4/10
1000/1000 [==============================] - 0s 90us/sample - loss: 38515.1085 - categorical_accuracy: 0.0880 - val_loss: 37862.3223 - val_categorical_accuracy: 0.1000
Epoch 5/10
1000/1000 [==============================] - 0s 90us/sample - loss: 44787.7728 - categorical_accuracy: 0.0960 - val_loss: 34271.9062 - val_categorical_accuracy: 0.1300
Epoch 6/10
1000/1000 [==============================] - 0

### tf.data 데이터셋을 사용한 훈련

In [12]:
# 예제 `Dataset` 객체를 만듭니다:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32)

# Dataset에서 `fit` 메서드를 호출할 때 `steps_per_epoch` 설정을 잊지 마세요.
model.fit(dataset, epochs=10, steps_per_epoch=30)

Train for 30 steps
Epoch 1/10
30/30 [==============================] - 0s 3ms/step - loss: 81012.3467 - categorical_accuracy: 0.0979
Epoch 2/10
 2/30 [=>............................] - ETA: 1s - loss: 84836.4766 - categorical_accuracy: 0.0750

* `steps_per_epoch` => 다음 에포크로 넘어가기 전 모델이 수행할 훈련 단계 횟수
* `Dataset`이 배치 데이터를 생성하기 때문에 `batch_size`가 필요하지 않음

In [13]:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32)

val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_labels))
val_dataset = val_dataset.batch(32)

model.fit(dataset, epochs=10,
          validation_data=val_dataset)

Epoch 1/10
32/32 [==============================] - 0s 5ms/step - loss: 91152.2454 - categorical_accuracy: 0.0810 - val_loss: 0.0000e+00 - val_categorical_accuracy: 0.0000e+00
Epoch 2/10
32/32 [==============================] - 0s 4ms/step - loss: 98027.9548 - categorical_accuracy: 0.0940 - val_loss: 68893.5801 - val_categorical_accuracy: 0.0900
Epoch 3/10
32/32 [==============================] - 0s 4ms/step - loss: 100850.9826 - categorical_accuracy: 0.0990 - val_loss: 103019.8691 - val_categorical_accuracy: 0.0600
Epoch 4/10
32/32 [==============================] - 0s 4ms/step - loss: 111894.9178 - categorical_accuracy: 0.0930 - val_loss: 89721.3672 - val_categorical_accuracy: 0.1000
Epoch 5/10
32/32 [==============================] - 0s 4ms/step - loss: 121002.6204 - categorical_accuracy: 0.1080 - val_loss: 144599.7734 - val_categorical_accuracy: 0.1200
Epoch 6/10
32/32 [==============================] - 0s 4ms/step - loss: 135408.1754 - categorical_accuracy: 0.0950 - val_loss: 1401